# Create or Update Model Serving Endpoint

Create or Update the deployed serving endpoints with a new model version.

* Make sure you've created online tables for all the required feature tables.
* Run this job on the workspace where you want to serve the model. 

In [ ]:
%pip install databricks-sdk>=0.38.0
%restart_python

In [0]:
dbutils.widgets.text("endpoint_name", defaultValue="")
dbutils.widgets.text("model_name", defaultValue="")
dbutils.widgets.text("model_version", defaultValue="")

In [0]:
ARGS = dbutils.widgets.getAll()

endpoint_name = ARGS["endpoint_name"]
model_name = ARGS["model_name"]
model_version = ARGS["model_version"]

{'endpoint_name': 'test-my-dab-endpoint', 'model_name': 'feature_store.sharing_bugbash_a7ce.fs_model', 'model_version': '2'}


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ServedEntityInput, EndpointCoreConfigInput
from databricks.sdk.errors import ResourceDoesNotExist

workspace = WorkspaceClient()

In [0]:
try:
    endpoint = workspace.serving_endpoints.get(name=endpoint_name)
except ResourceDoesNotExist as e:
    endpoint = None

if endpoint is None:
    workspace.serving_endpoints.create(
        name=endpoint_name,
        config=EndpointCoreConfigInput(
            served_entities=[
                ServedEntityInput(
                    entity_name=model_name,
                    entity_version=model_version,
                    scale_to_zero_enabled=True,
                    workload_size="Small",
                )
            ]
        ),
    )
    print(f"Created endpoint {endpoint_name}")
elif endpoint.pending_config is not None:
    print(f"A pending update for endpoint {endpoint_name} is being processed.")
elif (
    endpoint.config.served_entities[0].entity_name != model_name
    or endpoint.config.served_entities[0].entity_version != model_version
):
    # Update endpoint
    workspace.serving_endpoints.update_config(
        name=endpoint_name,
        served_entities=[
            ServedEntityInput(
                entity_name=model_name,
                entity_version=model_version,
                scale_to_zero_enabled=True,
                workload_size="Small",
            )
        ],
    )
    print(f"Updated endpoint {endpoint_name}")
else:
    print("Endpoint already up-to-date")

A pending update for endpoint test-my-dab-endpoint is being processed.
